# Atualização do Dataset e Uso para Validação do Modelo

## Status do Dataset

O dataset utilizado em nosso projeto foi **atualizado até outubro de 2024**. No entanto, ele contém registros de lutas que ocorrerão em **14 de dezembro de 2024**. Esses dados futuros serão utilizados para ilustrar a eficácia do modelo, permitindo uma avaliação mais robusta e realista de seu desempenho.


In [157]:
import pandas as pd
import numpy as np

Primeiramente, realizaremos a leitura dos dados das 13 lutas que serão analisadas. Para garantir que o modelo possa utilizá-los corretamente, aplicaremos o mesmo processo de tratamento e pré-processamento utilizado nos dados de treinamento, assegurando consistência e compatibilidade entre as amostras.

In [158]:
test_fights = pd.read_csv('data/upcoming.csv')
training_data = pd.read_csv('data/ufc-master.csv')

data = pd.concat([test_fights, training_data])

In [159]:
data = data.drop(columns=[
    'RedExpectedValue', 
    'BlueExpectedValue', 
    'EmptyArena', 
    'BMatchWCRank', 
    'RMatchWCRank',
    'RWFlyweightRank',
    'RWFeatherweightRank',
    'RWStrawweightRank',
    'RWBantamweightRank',
    'RHeavyweightRank',
    'RLightHeavyweightRank',
    'RMiddleweightRank',
    'RWelterweightRank',
    'RLightweightRank',
    'RFeatherweightRank',
    'RBantamweightRank',
    'RFlyweightRank',
    'RPFPRank',
    'BWFlyweightRank',
    'BWFeatherweightRank',
    'BWStrawweightRank',
    'BWBantamweightRank',
    'BHeavyweightRank',
    'BLightHeavyweightRank',
    'BMiddleweightRank',
    'BWelterweightRank',
    'BLightweightRank',
    'BFeatherweightRank',
    'BBantamweightRank',
    'BFlyweightRank',
    'BPFPRank',
    'FinishDetails',
    'FinishRoundTime',
    'RedDecOdds',
    'BlueDecOdds',
    'RSubOdds',
    'BSubOdds',
    'RKOOdds',
    'BKOOdds',
    'TotalFightTimeSecs',
    'Finish',
    'FinishRound',
    'Location',
    'Country',
    'BlueOdds',
    'RedOdds'
])




In [160]:
global_red_sig_str_mean = data['RedAvgSigStrLanded'].mean()
global_blue_sig_str_mean = data['BlueAvgSigStrLanded'].mean()

data['RedAvgSigStrLanded'] = (
    data['RedAvgSigStrLanded']
    .fillna(data.groupby('RedFighter')['RedAvgSigStrLanded'].transform('mean'))
    .fillna(global_red_sig_str_mean)
)

data['BlueAvgSigStrLanded'] = (
    data['BlueAvgSigStrLanded']
    .fillna(data.groupby('BlueFighter')['BlueAvgSigStrLanded'].transform('mean'))
    .fillna(global_blue_sig_str_mean)
)

In [161]:
global_red_sig_str_pctmean = data['RedAvgSigStrPct'].mean()
global_blue_sig_str_pctmean = data['BlueAvgSigStrPct'].mean()

data['RedAvgSigStrPct'] = (
    data['RedAvgSigStrPct']
    .fillna(data.groupby('RedFighter')['RedAvgSigStrPct'].transform('mean'))
    .fillna(global_red_sig_str_pctmean)
)

data['BlueAvgSigStrPct'] = (
    data['BlueAvgSigStrPct']
    .fillna(data.groupby('BlueFighter')['BlueAvgSigStrPct'].transform('mean'))
    .fillna(global_blue_sig_str_pctmean)
)

In [162]:
global_red_sub_att_mean = data['RedAvgSubAtt'].mean()
global_blue_sub_att_mean = data['BlueAvgSubAtt'].mean()

data['RedAvgSubAtt'] = (
    data['RedAvgSubAtt']
    .fillna(data.groupby('RedFighter')['RedAvgSubAtt'].transform('mean'))
    .fillna(global_red_sub_att_mean)
)

data['BlueAvgSubAtt'] = (
    data['BlueAvgSubAtt']
    .fillna(data.groupby('BlueFighter')['BlueAvgSubAtt'].transform('mean'))
    .fillna(global_blue_sub_att_mean)
)

In [163]:
global_red_td_mean = data['RedAvgTDLanded'].mean()
global_blue_td_mean = data['BlueAvgTDLanded'].mean()

data['RedAvgTDLanded'] = (
    data['RedAvgTDLanded']
    .fillna(data.groupby('RedFighter')['RedAvgTDLanded'].transform('mean'))
    .fillna(global_red_td_mean)
)

data['BlueAvgTDLanded'] = (
    data['BlueAvgTDLanded']
    .fillna(data.groupby('BlueFighter')['BlueAvgTDLanded'].transform('mean'))
    .fillna(global_blue_td_mean)
)

In [164]:
global_red_td_pct_mean = data['RedAvgTDPct'].mean()
global_blue_td_pct_mean = data['BlueAvgTDPct'].mean()

data['RedAvgTDPct'] = (
    data['RedAvgTDPct']
    .fillna(data.groupby('RedFighter')['RedAvgTDPct'].transform('mean'))
    .fillna(global_red_td_pct_mean)
)

data['BlueAvgTDPct'] = (
    data['BlueAvgTDPct']
    .fillna(data.groupby('BlueFighter')['BlueAvgTDPct'].transform('mean'))
    .fillna(global_blue_td_pct_mean)
)

In [165]:
global_blue_stance_mode = data['BlueStance'].mode()[0]  

data['BlueStance'] = (
    data['BlueStance']
    .fillna(data.groupby('BlueFighter')['BlueStance'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
    .fillna(global_blue_stance_mode)
)


Separando as lutas de teste

In [166]:
test_fights= data.merge(
    test_fights[['RedFighter', 'BlueFighter', 'Date']],
    on=['RedFighter', 'BlueFighter','Date'],
    how='inner'
)

In [167]:
merged = data.merge(
    test_fights[['RedFighter', 'BlueFighter', 'Date']],
    on=['RedFighter', 'BlueFighter', 'Date'],
    how='left',
    indicator=True
)

data = merged[merged['_merge'] == 'left_only']

Calculo do elo para as amostras de treino

In [168]:
data = data.sort_values(by='Date').reset_index(drop=True)

In [169]:
def update_elo(winner_elo, loser_elo, K=32):
    """
    Atualiza o Elo após uma luta.
    K = Fator de peso (quanto maior, mais rápido a pontuação muda).
    """
    expected_win = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    change = K * (1 - expected_win)  
    
    new_winner_elo = winner_elo + change
    new_loser_elo = loser_elo - change
    
    return new_winner_elo, new_loser_elo

In [170]:
elo_dict = {}
initial_elo = 1200  

all_fighters = pd.concat([data['RedFighter'], data['BlueFighter']]).unique()

# Inicializar o Elo para todos os lutadores
for fighter in all_fighters:
    elo_dict[fighter] = initial_elo

# Função para atualizar o Elo
def update_elo(winner_elo, loser_elo, K=32):
    expected_win = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    change = K * (1 - expected_win)
    return winner_elo + change, loser_elo - change

red_elo_current = []
blue_elo_current = []

for idx, row in data.iterrows():
    red = row['RedFighter']
    blue = row['BlueFighter']
    winner = row['Winner']
    
    # Elo atual antes da luta (armazenado no dicionário)
    red_elo = elo_dict[red]
    blue_elo = elo_dict[blue]
    
    # Atualizar Elo com base no resultado
    if winner == 'Red':
        new_red, new_blue = update_elo(red_elo, blue_elo)
    elif winner == 'Blue':
        new_blue, new_red = update_elo(blue_elo, red_elo)
    else:
        new_red = red_elo
        new_blue = blue_elo
    
    elo_dict[red] = new_red
    elo_dict[blue] = new_blue
    
    red_elo_current.append(new_red)
    blue_elo_current.append(new_blue)

data['RedEloCurrent'] = red_elo_current
data['BlueEloCurrent'] = blue_elo_current

In [171]:
import numpy as np

data['EloDiff'] = np.abs(data['RedEloCurrent'] - data['BlueEloCurrent'])

Atribui o elo as lutas teste

In [172]:
import pandas as pd

red_fighters = data[['RedFighter', 'RedEloCurrent', 'Date']].rename(
    columns={'RedFighter': 'Fighter', 'RedEloCurrent': 'EloCurrent'}
)
blue_fighters = data[['BlueFighter', 'BlueEloCurrent', 'Date']].rename(
    columns={'BlueFighter': 'Fighter', 'BlueEloCurrent': 'EloCurrent'}
)

all_fighters = pd.concat([red_fighters, blue_fighters], ignore_index=True)

all_fighters['Date'] = pd.to_datetime(all_fighters['Date'])  
latest_elos = (
    all_fighters
    .sort_values('Date')  
    .groupby('Fighter')    
    .tail(1)               
    .set_index('Fighter')['EloCurrent']
    .to_dict()
)

test_fights['RedEloCurrent'] = test_fights['RedFighter'].map(latest_elos)
test_fights['BlueEloCurrent'] = test_fights['BlueFighter'].map(latest_elos)


test_fights['RedEloCurrent'] = test_fights['RedEloCurrent'].fillna(initial_elo)
test_fights['BlueEloCurrent'] = test_fights['BlueEloCurrent'].fillna(initial_elo)
test_fights['EloDiff'] = np.abs(test_fights['RedEloCurrent'] - test_fights['BlueEloCurrent'])

In [173]:
data = pd.concat([data, test_fights])

In [174]:
data = data.drop(columns=['Winner'])

In [175]:
from sklearn.preprocessing import QuantileTransformer

colunas_a_normalizar = [
    'BlueHeightCms', 'BlueReachCms', 'BlueWeightLbs',
    'RedHeightCms', 'RedReachCms', 'RedWeightLbs',
    'RedEloCurrent', 'BlueEloCurrent', 'EloDiff'
]

transformer = QuantileTransformer()
data[colunas_a_normalizar] = transformer.fit_transform(data[colunas_a_normalizar])


In [176]:
data = data.drop(columns=['_merge'])

In [177]:
object_cols = data.select_dtypes(include=['object']).columns.difference(['RedFighter', 'BlueFighter', 'Date'])


data = pd.get_dummies(data, columns=object_cols)

In [178]:
test_fights= data.merge(
    test_fights[['RedFighter', 'BlueFighter', 'Date']],
    on=['RedFighter', 'BlueFighter','Date'],
    how='inner'
)

In [179]:
from sklearn.preprocessing import LabelEncoder

lencoder = LabelEncoder()

all_fighters = pd.concat([data['RedFighter'], data['BlueFighter']]).unique()

lencoder.fit(all_fighters)

test_fights['RedFighter'] = lencoder.transform(test_fights['RedFighter'])
test_fights['BlueFighter'] = lencoder.transform(test_fights['BlueFighter'])

data['RedFighter'] = lencoder.transform(data['RedFighter'])
data['BlueFighter'] = lencoder.transform(data['BlueFighter'])


In [180]:
test_fights_tratados = test_fights.drop(columns=['Date'])

In [181]:
dados_treinamento = pd.read_csv('data/dados_tratados.csv')
dados_treinamento = dados_treinamento.drop(columns=['Winner'])
test_fights_tratados = test_fights_tratados[dados_treinamento.columns]

In [182]:
raw_data = pd.read_csv('data/upcoming.csv')

## Predicao em lutas sem resultado

In [183]:
import os
import joblib
import glob

def carregar_modelos(diretorio='models'):
    modelos = {}
    caminho_padro = os.path.join(diretorio, '*.pkl')
    
    for arquivo in glob.glob(caminho_padro):
        nome_modelo = os.path.splitext(os.path.basename(arquivo))[0]
        modelo = joblib.load(arquivo)  
        modelos[nome_modelo] = modelo
        print(f"Modelo '{nome_modelo}' carregado com sucesso!")
    
    return modelos

modelos_carregados = carregar_modelos()


Modelo 'xgboost' carregado com sucesso!
Modelo 'random_forest' carregado com sucesso!
Modelo 'mlp' carregado com sucesso!
Modelo 'logistic_regression' carregado com sucesso!


Como o modelo com a melhor acuracia media ao utilizar o cross-validation foi a Regressao Logistica, sera ela que iremos utilizar para as predicoes

### Predicao de Lutas

In [184]:
for i in range(len(raw_data)):
    X = dados_treinamento.iloc[[i], :].values  
    resultados = modelos_carregados['logistic_regression'].predict_proba(X)

    print(f"Luta {i + 1} - {raw_data['RedFighter'][i]} vs {raw_data['BlueFighter'][i]}")
    print(f"Probabilidade do vencedor ser {raw_data['RedFighter'][i]}: {resultados[0][1] * 100:.2f}%")
    print(f"Probabilidade do vencedor ser {raw_data['BlueFighter'][i]}: {resultados[0][0] * 100:.2f}%\n")


Luta 1 - Colby Covington vs Joaquin Buckley
Probabilidade do vencedor ser Colby Covington: 45.24%
Probabilidade do vencedor ser Joaquin Buckley: 54.76%

Luta 2 - Cub Swanson vs Billy Quarantillo
Probabilidade do vencedor ser Cub Swanson: 80.70%
Probabilidade do vencedor ser Billy Quarantillo: 19.30%

Luta 3 - Manel Kape vs Bruno Silva
Probabilidade do vencedor ser Manel Kape: 47.35%
Probabilidade do vencedor ser Bruno Silva: 52.65%

Luta 4 - Vitor Petrino vs Dustin Jacoby
Probabilidade do vencedor ser Vitor Petrino: 95.45%
Probabilidade do vencedor ser Dustin Jacoby: 4.55%

Luta 5 - Adrian Yanez vs Daniel Marcos
Probabilidade do vencedor ser Adrian Yanez: 67.03%
Probabilidade do vencedor ser Daniel Marcos: 32.97%

Luta 6 - Navajo Stirling vs Tuco Tokkos
Probabilidade do vencedor ser Navajo Stirling: 95.61%
Probabilidade do vencedor ser Tuco Tokkos: 4.39%

Luta 7 - Michael Johnson vs Ottman Azaitar
Probabilidade do vencedor ser Michael Johnson: 75.69%
Probabilidade do vencedor ser Ottma

### Analise das Predicoes

* Luta 1: Acerto do modelo ✅

* Luta 2: Acerto do modelo ✅

* Luta 3: Erro do modelo ❌

* Luta 4: Acerto do modelo ✅

* Luta 5: Erro do modelo ❌

* Luta 6: Acerto do modelo ✅

* Luta 7: Acerto do modelo ✅

* Luta 8: Acerto do modelo ✅

* Luta 9: Acerto do modelo ✅

* Luta 10: Erro do modelo ❌

* Luta 11: Acerto do modelo ✅

* Luta 12: Acerto do modelo ✅

* Luta 13: Erro do modelo ❌

# Análise do Modelo de Previsão para Apostas no MMA

## Resultados Obtidos

Após testar o modelo neste pequeno conjunto de dados, obtivemos uma **acurácia média de 70%**. Apesar da queda na acurácia em comparação com os resultados obtidos durante o **cross-validation**, isso pode estar relacionado ao tamanho reduzido do conjunto de testes, que pode sofrer com a **variabilidade devido ao número limitado de amostras**. Conjuntos menores são mais suscetíveis a flutuações estatísticas, o que pode distorcer a avaliação do desempenho real do modelo.

Mesmo assim, o modelo se mostra **mais eficaz** do que simplesmente seguir a **odd mais baixa** (considerada a mais segura) para se apostar, uma técnica que tem uma eficácia média de cerca de **65%** das ocasiões, conforme observado em análises históricas.

---

## A Importância de Machine Learning no MMA

Em suma, apesar da **imprevisibilidade presente no mundo do MMA**, onde fatores como lesões, imprevistos durante as lutas e a volatilidade dos atletas muitas vezes contradizem as probabilidades estimadas, o uso de técnicas de **machine learning** permite trazer um pouco de **previsibilidade e embasamento científico** a este ambiente caótico.

A aplicação de modelos preditivos não apenas supera estratégias simplistas, como seguir as odds mais baixas, mas também abre caminho para análises mais sofisticadas, considerando variáveis como:

- **Estilo de luta** (grappler vs. striker)
- **Histórico de desempenho** dos atletas

---

## Limitações e Considerações Finais

No entanto, é importante ressaltar que **nenhum modelo é infalível**, especialmente em um domínio tão dinâmico quanto o MMA. A combinação de:

1. **Dados históricos**
2. **Análise estatística**
3. **Entendimento contextual** das lutas

pode maximizar a eficácia das previsões, mas a **incerteza** sempre será uma parte intrínseca desse esporte. Portanto, o uso de machine learning deve ser visto como uma **ferramenta complementar**, capaz de auxiliar na tomada de decisões, mas não como uma garantia absoluta de sucesso.

---